In [67]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

Defs:

In [68]:
# def for checking str in every file
def find_str_in_all_files(str_to_find):
    in_dict = {}
    for count in range(len(files_df)):
        str_to_find_lower = str_to_find.strip().lower()
        file = files_df.loc[count,'files']
        data = pd.read_excel(f'{files_df.loc[count,"folder"]}\\{file}', sheet_name=sheet_name)
        try:
            str_index = data[data.apply(lambda row: row.astype(str).str.lower().str.contains(str_to_find_lower).any(), axis=1)].index.item()
            in_dict[file] = str_index
        except:
            continue
    return files_df.shape[0], len(in_dict.keys()), in_dict

# def for finfing str index
def find_id_str(str_to_find):
    str_to_find_lower = str_to_find.strip().lower()
    id_str = new_data[new_data.apply(lambda row: row.astype(str).str.lower().str.contains(str_to_find_lower).any(), axis=1)].index.item()
    return id_str

def diff_by_cols(data_col_names, data_merge, count_start_col=1, data_for_cols=True):
    
    if data_for_cols==False:
        cols_list = ['Value']
    else:
        # cols lisr without name col (first)
        cols_list = data_col_names.columns.tolist()[count_start_col:]

    for col in cols_list:
        # choose only cols with previous col in name
        col_cols_list = [x for x,y in zip(data_merge.columns,data_merge.columns.str.contains(col)) if y==True]
        assert len(col_cols_list) == 2
        # print diff
        data_merge[f'{col}_diff'] = -1*(data_merge[col_cols_list[0]] - data_merge[col_cols_list[1]])
        # print pct diff
        data_merge[f'{col}_pct_diff'] = 0 
        # condition = abs(data_merge[[col_cols_list[0],col_cols_list[1]]]).max(axis=1)!=0
        # data_merge.loc[condition, f'{col}_pct_diff'] = data_merge.loc[condition,f'{col}_diff'] / abs(data_merge.loc[condition,[col_cols_list[0],col_cols_list[1]]]).max(axis=1)
        condition = data_merge[col_cols_list[0]]!=0
        data_merge.loc[condition, f'{col}_pct_diff'] = abs(data_merge.loc[condition,f'{col}_diff'] / data_merge.loc[condition,col_cols_list[0]])
        data_merge[f'{col}_pct_diff'] = np.sign(data_merge[f'{col}_diff']) * data_merge[f'{col}_pct_diff']

    return data_merge

def del_null_data(data):
    list_to_del = [np.nan, '0', 0]
    # del empty rows
    data = data[(~data.isin(list_to_del)).any(axis=1)]
    # del empty cols
    cols_list = (~data.isin(list_to_del)).any(axis=0)
    cols_list = cols_list[cols_list==True].index.tolist()
    data = data[cols_list]
    return data

def data_to_parts(data,need_to_name=False):
    string_to_find = 'TOTAL'
    string_to_find = string_to_find.strip().lower()
    try:
        assert data.iloc[0,5].strip().lower() == string_to_find
        part_col = 6
    except:
        part_col = 5
    asserts_part = data.iloc[:,:part_col]
    asserts_part.columns = ['name'] + asserts_part.iloc[0,:].tolist()[1:]
    if need_to_name == True:
        asserts_part['class_name_1'] = 'Assets'
        asserts_part = asserts_part.iloc[2:,:]
    else:
        asserts_part = asserts_part.iloc[1:,:]

    liabilities_part = data.iloc[:,part_col:part_col+part_col]
    liabilities_part.columns = ['name'] + liabilities_part.iloc[0,:].tolist()[1:]
    if need_to_name == True:
        liabilities_part['class_name_1'] = 'Liabilities and equity'
        liabilities_part = liabilities_part.iloc[2:,:]
    else:
        liabilities_part = liabilities_part.iloc[1:,:]

    data = pd.concat([asserts_part,liabilities_part],axis=0).reset_index(drop=True)
    return data

Variables

In [69]:
print_to_excel = True

sheet_name = 'FX Exposure_Balance Sheet'
month = ['nov','dec']
month = month[1]

In [70]:
folders = [x for x in os.listdir() if month in x]
files_list = []
folders_for_df = []
for count in range(len(folders)):
    files_folder_list = os.listdir(f'{folders[count]}')
    files_folder_list = [x for x in files_folder_list if '~' not in x]
    files_list += files_folder_list
    folders_for_df += [folders[count] for x in range(len(files_folder_list))]
files_list

['BRFHRG FX Exposure 02.12.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFTO FX Exposure Model 02.12.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 02.12.2024 - With Pnl & Tax Credit.xlsx',
 'BRFHRG FX Exposure 03.12.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFTO FX Exposure Model 03.12.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 03.12.2024 - With Pnl & Tax Credit.xlsx']

In [71]:
files_df = pd.DataFrame({'files':files_list, 'folder':folders_for_df})
files_df['day'] = files_df.files.str.extract(r'([0-9]{2}.[0-9]{2}.[0-9]{4})')
files_df['company'] = files_df.files.str.extract(r'(BR\w* )')
files_df

,files,folder,day,company
0,BRFHRG FX Exposure 02.12.2024 - With pnl Tax ...,2 dec,02.12.2024,BRFHRG
1,BRFTO FX Exposure Model 02.12.2024 - With Pnl ...,2 dec,02.12.2024,BRFTO
2,BRSFT FX Exposure Model 02.12.2024 - With Pnl ...,2 dec,02.12.2024,BRSFT
3,BRFHRG FX Exposure 03.12.2024 - With pnl Tax ...,3 dec,03.12.2024,BRFHRG
4,BRFTO FX Exposure Model 03.12.2024 - With Pnl ...,3 dec,03.12.2024,BRFTO
5,BRSFT FX Exposure Model 03.12.2024 - With Pnl ...,3 dec,03.12.2024,BRSFT


In [72]:
files_df_merge = files_df.merge(files_df, on='company', suffixes=['_old','_new'])
files_df_couple = files_df_merge[(files_df_merge.day_old!=files_df_merge.day_new)&(files_df_merge.day_old<files_df_merge.day_new)]
files_df_couple = files_df_couple.reset_index(drop=True)
files_df_couple

,files_old,folder_old,day_old,company,files_new,folder_new,day_new
0,BRFHRG FX Exposure 02.12.2024 - With pnl Tax ...,2 dec,02.12.2024,BRFHRG,BRFHRG FX Exposure 03.12.2024 - With pnl Tax ...,3 dec,03.12.2024
1,BRFTO FX Exposure Model 02.12.2024 - With Pnl ...,2 dec,02.12.2024,BRFTO,BRFTO FX Exposure Model 03.12.2024 - With Pnl ...,3 dec,03.12.2024
2,BRSFT FX Exposure Model 02.12.2024 - With Pnl ...,2 dec,02.12.2024,BRSFT,BRSFT FX Exposure Model 03.12.2024 - With Pnl ...,3 dec,03.12.2024


Counting

In [73]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_Brazil_monitoring_{month}.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl', mode='w') as writer:
        files_df_couple.to_excel(writer, index=False, sheet_name='all_files')

list_to_del = [np.nan, '0', 0]
for string_id in files_df_couple.index:
    # try:
    # read old data
    old_path = f'{files_df_couple.loc[string_id, "folder_old"]}\\{files_df_couple.loc[string_id, "files_old"]}'
    old_data = pd.read_excel(old_path, sheet_name=sheet_name)
    old_data_date = files_df_couple.loc[string_id, "day_old"]
    print(old_path)

    # read new data
    new_path = f'{files_df_couple.loc[string_id, "folder_new"]}\\{files_df_couple.loc[string_id, "files_new"]}'
    new_data = pd.read_excel(new_path, sheet_name=sheet_name)
    new_data_date = files_df_couple.loc[string_id, "day_new"]
    print(new_path)

    company_name = files_df_couple.loc[string_id, "company"]

    count_data_date = 0
    for data_date in [old_data, new_data]:
        
        count_data_date += 1



        ### DATA_1
        # 1)	сначала видеть изменение общей позиции, суммарной день ко 
        # 2)	затем видеть изменение on-Balance и Off-Balance
        start_str = 'USD Equivalent'
        end_str = 'Net Exposure Balance BRL'

        data_1 = data_date.loc[find_id_str(start_str):find_id_str(end_str),]
        data_1 = del_null_data(data_1)
        data_1 = data_1.iloc[:,:3].reset_index(drop=True)
        data_1.columns = ['name'] + data_1.iloc[0,1:].values.tolist()

        data_1 = data_1[~data_1.name.isna()]
        data_1_melt = data_1.melt(id_vars='name', value_vars=data_1.columns.tolist()[1:],var_name='Exposure currency type', value_name='Value')
        
        data_11 = data_1_melt[data_1_melt.name.str.lower().str.contains('net')]
        data_11.columns = ['Total net exposure'] + data_11.columns.tolist()[1:]
        
        data_12 = data_1_melt[~data_1_melt.name.str.lower().str.contains('net')]
        data_12.columns = ['Total exposure split by Balance/Off balance'] + data_12.columns.tolist()[1:]

        ### DATA_3
        # 3)	затем видеть изменение итоговых балансовых статей:
        start_str = 'USD denominated'
        end_str = 'Total assets'
        data_3 = data_date.loc[[find_id_str(start_str),find_id_str(end_str)],]
        data_3 = del_null_data(data_3)

        data_3 = data_to_parts(data_3)

        col_1_name_data_3 = 'Total balance and off-balance items'

        data_3_tr = data_3.transpose()
        # rename cols and drop col name string
        data_3_tr.columns = data_3_tr.iloc[0,:]
        data_3_tr = data_3_tr.iloc[1:,:]
        data_3_tr['name'] = data_3_tr.index
        data_3_tr = data_3_tr.reset_index(drop=True)
        # to parts
        assets_col = [x for x in data_3_tr.columns.tolist() if 'assets' in x][0]
        liabilities_col = [x for x in data_3_tr.columns.tolist() if 'liabilities' in x][0]


        assets_data = data_3_tr[['name',assets_col]]
        assets_data.columns = [col_1_name_data_3, 'Value']
        assets_data['Balance item class'] = 'Assets'
        liabilities_data = data_3_tr[['name',liabilities_col]]
        liabilities_data.columns = [col_1_name_data_3, 'Value']
        liabilities_data['Balance item class'] = 'Liabilities and equity'

        data_3_tr = pd.concat([assets_data,liabilities_data], axis=0)
        data_3_tr = data_3_tr.iloc[:,[2,0,1]]

        data_3 = data_3_tr

        ### DATA_4
        # 4)	затем видеть изменения внебалансовых статей (их немного)
        start_str = 'Total assets'
        end_str = 'Total Off Balance Assets'
        pre_data_4 = data_date.loc[find_id_str(start_str):find_id_str(end_str),]

        start_str = 'Assets'
        header_str = 'USD denominated'
        start_id = pre_data_4.loc[pre_data_4['Unnamed: 1']=='Assets',:].index.item()
        data_4 = pre_data_4.loc[start_id:]

        header_df = data_date.loc[find_id_str(header_str):find_id_str(header_str)+1,]
        data_4 = pd.concat([header_df,data_4],axis=0)
        data_4 = del_null_data(data_4)

        data_4 = data_to_parts(data_4, True)

        data_4 = data_4[~data_4.iloc[:,0].isna()].reset_index(drop=True)
        # del empty cols
        cols_list = (~data_4.isin(list_to_del)).any(axis=0)
        cols_list = cols_list[cols_list==True].index.tolist()
        data_4 = data_4[cols_list]

        data_4 = data_4[['class_name_1'] + data_4.columns.tolist()[:-1]]

        # del string
        str_to_find = 'Industrialization'
        str_to_find = str_to_find.strip().lower()
        data_4 = data_4[~data_4.name.str.lower().str.contains(str_to_find)]

        # total to previous part
        str_to_find = 'total'

        data_to_3 = data_4[data_4.name.str.lower().str.contains(str_to_find)]
        data_to_3 = data_to_3.iloc[:,:3]
        data_to_3.columns = ['Balance item class', col_1_name_data_3, 'Value']
        data_to_3.iloc[:,1] = 'Off balance'

        data_3 = pd.concat([data_3,data_to_3],axis=0)

        #final 4 part
        data_4 = data_4[~data_4.name.str.lower().str.contains(str_to_find)]
        data_4 = data_4.iloc[:, :3]
        data_4.columns = ['Balance item class','Off- balance items','Value']

        ### DATA_5
        # 5)	затем видеть детальное изменение балансовых статей.
        start_str = 'USD denominated'
        end_str = 'Total assets'
        data_5 = data_date.loc[find_id_str(start_str):find_id_str(end_str)-1,]
        data_5 = del_null_data(data_5)

        data_5 = data_to_parts(data_5, True)

        data_5 = data_5[~data_5.iloc[:,0].isna()].reset_index(drop=True)

        class_name_list = ['Current assets','Non-current assets','Current liabilities','Non-current liabilities','Equity']
        class_name_list_lower = list(map(lambda x: x.strip().lower(), class_name_list))
        data_5['class_name_2'] = np.nan
        condition = data_5.name.str.strip().str.lower().isin(class_name_list_lower)
        data_5.loc[condition, 'class_name_2'] = data_5.loc[condition, 'name']
        data_5['class_name_2'] = data_5['class_name_2'].fillna(method='ffill')
        data_5 = data_5[(data_5.name!=data_5.class_name_2)|(data_5.name=='Equity')].reset_index(drop=True)

        data_5 = data_5[['class_name_1', 'class_name_2'] + data_5.columns.tolist()[:-2]]

        data_5_melt = data_5.melt(id_vars=data_5.columns.tolist()[:3], value_vars=data_5.columns.tolist()[3:], value_name='Value', var_name='Exposure currency type')
        data_5_melt = data_5_melt.iloc[:,1:]
        data_5_melt.columns = ['Balance item class','Balance item name'] + data_5_melt.columns.tolist()[2:]
        data_5 = data_5_melt
        

        if count_data_date == 1:
            data_11_old = data_11
            data_12_old = data_12
            data_3_old = data_3
            data_4_old = data_4
            data_5_old = data_5
        if count_data_date == 2:
            data_11_new = data_11
            data_12_new = data_12
            data_3_new = data_3
            data_4_new = data_4
            data_5_new = data_5

    # concat old and new data
    ### data_1
    data_11_merge = data_11_old.merge(data_11_new, how='outer', on=['Total net exposure','Exposure currency type'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_11_merge = diff_by_cols(data_11_new,data_11_merge,2,False)

    data_12_merge = data_12_old.merge(data_12_new, how='outer', on=['Total exposure split by Balance/Off balance','Exposure currency type'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_12_merge = diff_by_cols(data_12_new,data_12_merge,2,False)

    ### data_3
    data_3_merge = data_3_old.merge(data_3_new, how='outer', on=['Balance item class','Total balance and off-balance items'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_3_merge = diff_by_cols(data_3,data_3_merge, data_for_cols=False)

    ### data_4
    data_4_merge = data_4_old.merge(data_4_new, how='outer', on=['Balance item class','Off- balance items'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_4_merge = data_4_merge.fillna(0)
    data_4_merge = diff_by_cols(data_4,data_4_merge,2,data_for_cols=False)

    ### data_5
    data_5_merge = data_5_old.merge(data_5_new, how='outer', on=['Balance item class','Balance item name','Exposure currency type'], suffixes=[f'_{old_data_date}',f'_{new_data_date}'])
    data_5_merge = data_5_merge.fillna(0)
    data_5_merge = diff_by_cols(data_5,data_5_merge,3,data_for_cols=False)

    ### TO EXCEL
    if print_to_excel == True:
        with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            count_row = 0
            pd.DataFrame({'company':company_name},index=[0]).to_excel(writer, startcol=2, header=False, index=False, sheet_name=company_name, startrow=count_row)
            count_row += 2
            files_df_couple.loc[string_id:string_id,:].to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += 3
            data_11_merge.to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_11_merge) + 2
            data_12_merge.to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_12_merge) + 2
            data_3_merge.to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_3_merge) + 2
            data_4_merge.to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)
            count_row += len(data_4_merge) + 2
            data_5_merge.to_excel(writer, startcol=2, header=True, index=False, sheet_name=company_name, startrow=count_row)




    # except Exception as e:
    #     print(e, 'in string ',string_id)

2 dec\BRFHRG FX Exposure 02.12.2024 - With pnl  Tax Credit - New model.xlsx
3 dec\BRFHRG FX Exposure 03.12.2024 - With pnl  Tax Credit - New model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\176471140.py:144: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a f

2 dec\BRFTO FX Exposure Model 02.12.2024 - With Pnl & Tax Credit -New Model.xlsx
3 dec\BRFTO FX Exposure Model 03.12.2024 - With Pnl & Tax Credit -New Model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\176471140.py:144: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a f

2 dec\BRSFT FX Exposure Model 02.12.2024 - With Pnl & Tax Credit.xlsx
3 dec\BRSFT FX Exposure Model 03.12.2024 - With Pnl & Tax Credit.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asserts_part['class_name_1'] = 'Assets'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\107726799.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liabilities_part['class_name_1'] = 'Liabilities and equity'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12256\176471140.py:144: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a f

check

In [74]:
files_df_couple.company.unique().tolist()

['BRFHRG ', 'BRFTO ', 'BRSFT ']

In [75]:
### FORMAT
if print_to_excel == True:
    
    for company_name in files_df_couple.company.unique().tolist():
        
        # open file
        wb = openpyxl.load_workbook(Output_file)
        ws = wb[company_name]

        # Weight of olumns
        ws.column_dimensions['A'].width = 4
        ws.column_dimensions['B'].width = 4
        ws.column_dimensions['C'].width = 37
        ws.column_dimensions['D'].width = 41
        ws.column_dimensions['E'].width = 25
        ws.column_dimensions['F'].width = 16
        ws.column_dimensions['G'].width = 16
        ws.column_dimensions['H'].width = 16
        ws.column_dimensions['I'].width = 16

        #alignment 
        from openpyxl.styles import Alignment
        for row in ws['C1:I200']:
            for cell in row:
                cell.alignment = Alignment(horizontal='left')
        for row in ws['E6:I200']:
            for cell in row:
                cell.alignment = Alignment(horizontal='right')
                cell.number_format = '# ### ###.00'

        # close file
        wb.save(Output_file)
        wb.close()
    